# Documentation:
    https://keras-rl.readthedocs.io/en/latest/agents/dqn/
    https://gym.openai.com/envs/Pendulum-v0/
    https://gym.openai.com/envs/#classic_control

In [2]:
import gym
import random
import tensorflow as tf
import numpy as np
from IPython import display

print(tf.__version__)

print('Imported ...')

2.3.0
Imported ...


# 1 Test the environement:
    To understand the environroment :
    The observations are the pedulum head cartesian coordiantes (x = 1*cos(theta),y = 1*sin(theta)) and the angular velocity thetadot
    The action is torque between -2 and 2 N.m
    The reward is a continues cost, cost = 0 if and only if (theta, thetadot, u) = (0, 0, 0),  it is negatives otherwise
    The env will nevet be DONE ! so the variable done is always False
    There is no info about the env

In [3]:
env = gym.make('Pendulum-v0')
nb_actions = env.action_space.shape[0] 
nb_states = env.observation_space.shape[0]
print('actions number: {} | states number: {}\n'.format(nb_actions, nb_states))

print('Done...')

actions number: 1 | states number: 3

Done...


In [4]:
episodes = 5

for episode in range(episodes):
    env.reset()
    done = False
    score = 0
    step = 0
    for step in range(100):
        action =  random.uniform(-2,2) #random.choice(range(actions))
        env.step([action])
        #score += reward
        step +=1
        print('Episode: ', episode, ' | Steps: ', step,  ' | Score: ', score)
        display.clear_output(wait=True)
        env.render()
        
        
env.close()    
print('finished...')    

finished...


In [5]:
env.close()

# 2 Make a Deep Learning Model with Keras:

In [8]:
!pip install keras
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

print('Done...')

Done...


In [9]:
def build_model(nb_states, nb_actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, nb_states)))
    model.add(Dense(24,activation = 'relu'))
    model.add(Dense(24,activation = 'relu'))
    model.add(Dense(nb_actions, activation = 'linear'))
    return model
print('finished...')

finished...


In [10]:
model = build_model(nb_states,nb_actions)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                96        
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 25        
Total params: 721
Trainable params: 721
Non-trainable params: 0
_________________________________________________________________


# 3 Build Agent with Keras-RL

In [11]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

print('finished...')

finished...


In [14]:
def build_agent(nb_actions, model):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length = 1)
    agent = DQNAgent(policy =  policy, memory = memory,
                     nb_actions = nb_actions, model= model, 
                     nb_steps_warmup = 10 , target_model_update = 1e-1
                    )
    return agent


agent = build_agent(nb_actions, model)
agent.compile(Adam(lr=1e-3), metrics=['mae'])
    
print('finished ...')

finished ...


# 4 Train the Agent:

In [15]:
agent.fit(env, nb_steps = 50000, visualize = False, verbose =1)

print('done...')

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 9:35 - reward: -0.6712

/home/nadourh/anaconda3/lib/python3.8/site-packages/rl/memory.py:40: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 43s 4ms/step - reward: -6.3187
50 episodes - episode_reward: -1263.739 [-1944.760, -501.629] - loss: 391.638 - mae: 7.060 - mean_q: -372.397

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: -6.1340
50 episodes - episode_reward: -1226.801 [-1905.561, -503.462] - loss: 465.473 - mae: 7.800 - mean_q: -424.901

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 46s 5ms/step - reward: -6.4161
50 episodes - episode_reward: -1283.211 [-1883.966, -377.590] - loss: 471.673 - mae: 8.056 - mean_q: -419.822

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: -6.0136
50 episodes - episode_reward: -1202.711 [-1936.686, -502.395] - loss: 470.574 - mae: 7.611 - mean_q: -421.528

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: -6.3230
done, took 237.411 seconds

Note: I changed the line 180 in the line of the file below :
gedit ~/anaconda3/envs/nadour_ai/lib/python3.8/site-packages/tensorflow/python/keras/engine/training_v1.py

# 5 Test the Agent:

In [43]:
scors = agent.test(env, nb_episodes = 100, visualize = False)
print(np.mean(scors.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 176.000, steps: 176
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [54]:
env.close()

# 6 save the agent model:

In [47]:
agent.save_weights('CartPole-v0-QAgent.h5')

# References:
    Playing Atari with Deep Reinforcement Learning, Mnih et al., 2013
    Human-level control through deep reinforcement learning, Mnih et al., 2015
    Deep Reinforcement Learning with Double Q-learning, van Hasselt et al., 2015
    Dueling Network Architectures for Deep Reinforcement Learning, Wang et al., 2016